## **2. 거시경제 데이터 수집**

### **2-1. Ecos API**

In [ ]:
import datetime
import requests
import pandas as pd
from bs4 import BeautifulSoup

#한국은행 ECOS API를 사용하기 위해서는 개인 인증키 발급을 받아야 한다
#발급을 받으면 아래 private_api_key = "본인의 API키 입력하기" 위치에 개인키를 넣고, 각종 거시경제 데이터를 수집 할 수 있다
#https://ecos.bok.or.kr/api/#/

private_api_key = #"개인 API 입력"

# ECOS 에서 데이터를 호출하는 함수를 만든다
def EcosDownload(Statcode, Freq, Begdate, Enddate, Subcode1, Subcode2, Subcode3):
    url = 'http://ecos.bok.or.kr/api/StatisticSearch/%s/xml/kr/1/100000/%s/%s/%s/%s/%s/%s/%s/'%(private_api_key, Statcode, Freq, Begdate, Enddate, Subcode1, Subcode2, Subcode3)
    raw = requests.get(url)
    xml = BeautifulSoup(raw.text,'xml')

    # Pandas 데이터프레임으로 전환한다
    raw_data = xml.find_all("row")
    date_list = []
    value_list = []

    for item in raw_data:
        value = item.find('DATA_VALUE').text.encode('utf-8')
        date_str = item.find('TIME').text
        value = float(value)
        date_list.append(datetime.datetime.strptime(date_str,'%Y%m'))
        value_list.append(value)

    df = pd.DataFrame(index = date_list)
    df['value'] = value_list

    return df

In [ ]:
금리 = EcosDownload('722Y001', 'M', '202202', '202206', '0101000','','')
금리.head()

,value
2022-02-01,1.25
2022-03-01,1.25
2022-04-01,1.50
2022-05-01,1.75
2022-06-01,1.75


In [ ]:
소비자물가지수 = EcosDownload('901Y009', 'M', '202111', '202206', '0', '', '')
소비자물가지수.head()

,value
2021-11-01,103.87
2021-12-01,104.04
2022-01-01,104.69
2022-02-01,105.30
2022-03-01,106.06


In [ ]:
부동산가격지수 = EcosDownload('901Y064', 'M', '202111', '202206', 'P65B','','')
부동산가격지수.head()

,value
2021-11-01,0.429
2021-12-01,0.418
2022-01-01,0.368
2022-02-01,0.347
2022-03-01,0.359


## **3. 기업 공시데이터 수집**

### **3-1. Dart API**

In [ ]:
pip install opendartreader

ERROR: Operation cancelled by user


In [ ]:
 # api_key : Dart에서 발급받은 API KEY
 # corp_code : 기업 고유번호
 # year : 보고서 기준년도
 # reprt_code : 11013(1분기), 11012(반기), 11014(3분기), 11011(사업)

In [ ]:
import OpenDartReader

# 객체 생성 (API KEY 지정)
# https://opendart.fss.or.kr/
# 접속 후 가입 - 오픈API이용현황에서 개인키 확인

api_key = # '개인키 입력'
dart = OpenDartReader(api_key)


SyntaxError: ignored

In [ ]:
# 공시정보
# 특정기업(삼성전자) 상장이후 모든 공시 목록 (5,600 건+)
dart.list('삼성전자') # 기업이름 혹은

In [ ]:
dart.list('005930') # 종목코드를 사용할 수 있음

In [ ]:
# 특정기업(삼성전자) 특정 날짜 이후 공시목록 (날짜에 다양한 포맷이 가능 2022, '2022-01-01', '20220101' )
dart.list('삼성전자', start='2022-01-01') # 2022-01-01 ~ 오늘
dart.list('005930', start='2022-01-01') # 2022-01-01 ~ 오늘

In [ ]:
# 특정기업(삼성전자) 특정 일자 범위(start~end)의 공시목록 (날짜에 다양한 포맷이 가능)
dart.list('005930', start='2022-04-28', end='2022-10-21')

In [ ]:
# 특정기업(삼성전자) 1999년~이후 모든 정기보고서 (정정된 공시포함)
dart.list('005930', start='1999-01-01', kind='A', final=False)

# 특정기업(삼성전자) 1999년~이후 모든 정기보고서 (최종보고서)
dart.list('005930', start='1999-01-01', kind='A')

# 2022-07-01 하루동안 모든 기업의 공시목록
dart.list(start='20200701', end='20200701')

# 2022-01-01 ~ 2022-01-10 모든 회사의 모든 공시목록 (3,139 건)
dart.list(start='2022-01-01', end='2022-01-10')

# 2022-01-01 ~ 2022-01-10 모든 회사의 모든 공시목록 (정정된 공시포함) (3,587 건)
dart.list(start='2022-01-01', end='2022-01-10', final=False)

# 2022-01-01~2022-03-30 모든 회사의 정기보고서 (corp를 특정 하지 않으면 최대 3개월) (2,352 건)
dart.list(start='2022-01-01', end='2022-03-30', kind='A')

In [ ]:
# 공시정보 - 기업개황
# 기업의 개황정보
dart.company('005930')

In [ ]:
# 회사명에 "삼성전자"가 포함된 회사들에 대한 개황정보
dart.company_by_name('삼성전자')

In [ ]:
# 1-3. 공시정보 - 공시서류원본파일
# 삼성전자 사업보고서 (2022년 반기사업보고서) 원문 텍스트
xml_text = dart.document('20220816001711')

In [ ]:
#xml_text
#주석 제거 시 공시서류원본파일 출력

In [ ]:
# 1-4. 공시정보 - 고유번호
# 종목코드로 고유번호 얻기
dart.find_corp_code('005930')

In [ ]:
# 기업명으로 고유번호 얻기
dart.find_corp_code('삼성전자')

https://github.com/FinanceData/OpenDartReader

## **4. 금융데이터 핸들링**

In [ ]:
# 판다스의 다양한 메서드를 활용해서 데이터 수집 시, 본인이 필요로 하는 형태로 데이터를 가공해서 활용 가능

In [ ]:
pip install finance_datareader

In [ ]:
import FinanceDataReader as fdr

In [ ]:
df_samsung = fdr.DataReader('005930', '2021','2023')
df_samsung.head()

In [ ]:
df_samsung.tail()

### 4-1. shift
shift : 인덱스 이동

*   전날/다음날 데이터를 Feature로 활용가능
*   모델링 시 중요 (주가 예측 모델링 시 Target Feature 처리)

데이터 작업을 하면서 다른 행과 비교분석이 필요할 경우 판다스의 shift 함수를 활용 가능

새로 만들어준 Open_shift 컬럼에 Open 값이 하나씩 내려옴

In [ ]:
df_samsung['Open_shift'] = df_samsung['Open'].shift(1)
df_samsung

*   응용: 당일종가-전일 종가 비교



In [ ]:
df_samsung['Change_Close'] = df_samsung['Close']-df_samsung['Close'].shift(1)
df_samsung

### 4.1.1 Change 값 직접 구해보기

In [ ]:
# Change = (종가 - 전일종가) - 1
# df_samsung["Change2"]
df_samsung["Change2"] = df_samsung["Close"] / df_samsung["Close"].shift(1) - 1

In [ ]:
# 불러온 값과 직접 구한 값이 같은지 확인하기
df_samsung[["Change","Change2"]]

### 4.1.2 pct_change 값 직접 구하기

*   pandas의 pct_change를 사용




In [ ]:
# pct_change = (종가/전일종가) - 1

df_samsung["pct_change"] = df_samsung["Close"].pct_change()
df_samsung.head()

# 특정 N일에 대한 수익률을 구하고 싶다면 pct_change(periods=N)을 입력 e.g.) 3일간의 수익률을 구하고 싶다면 N = 3

In [ ]:
df_samsung.tail()

In [ ]:
# "Change", "pct_change" 시각화 하기
import matplotlib.pyplot as plt

df_samsung[["Change", "pct_change"]].plot(figsize=(10,4))
plt.axhline(0, c="k", lw=1)

### 4-2. 이동평균

통계학에서 이동평균(=롤링평균)은 전체 데이터 집합의 여러 하위 집합에 대한 일련의 평균을 만들어 데이터 요소를 분석하는 계산이며,
이동산술평균(Moving Mean) 또는 롤링산술평균(Rolling Mean)이라고도 한다.

일련의 연속된 숫자와 고정된 부분 집합 크기가 주어지면, 이동 평균의 첫 번째 요소는 연속된 숫자의 첫 고정 부분 집합의 평균을 취하여 구한다. 그 다음, "앞으로 이동"하여 하위 집합을 변경한다. 즉, 부분 집합의 첫 번째 숫자를 제외하고 연속된 숫자의 다음 값을 포함시킨다.

### 4.2.1 rolling
### 4.2.2 expanding

In [ ]:
df_samsung['avgFiveDay'] = df_samsung['Close'].rolling(window=5).mean()
df_samsung.head(5)

In [ ]:
df_samsung['avgFiveDay'].plot(figsize=(15,5))
plt.show()

### 4.2.3 단순이동평균(SMA : Simple Moving Average)

In [ ]:
# 7일, 14일 단순 이동평균 구하기
df_samsung["SMA7"] = df_samsung["Close"].rolling(7).mean()
df_samsung["SMA14"] = df_samsung["Close"].rolling(14).mean()

In [ ]:
# 단순이동평균 시각화
df_samsung[["Close", "SMA7", "SMA14"]].plot(figsize=(10,4), title="Simple Moving Average")

### 4.2.4 누적이동평균(Cumulative Moving Average)

In [ ]:
# expanding을 사용하면 window가 시간의 흐름에 따라 증가
df_samsung["CMA7"] = df_samsung["Close"].expanding(7).mean()

In [ ]:
# 누적이동평균 시각화
df_samsung[["Close","CMA7"]].plot(figsize=(10,4), title="Cumulative Moving Average")

### 4.2.5 지수이동평균(Exponential Moving Average)
최근 자료에 가중치를 부여하여 최근 가격 변동을 잘 나타낸다.

In [ ]:
# 최근 자료에 가중치를 더 주도록 지수 이동평균을 구한다.
df_samsung["EWM_short"] = df_samsung["Close"].ewm(span=12).mean() # 12거래일로 span 지정
df_samsung["EWM_long"] = df_samsung["Close"].ewm(span=26).mean()
# MACD 지표 사용할 때 주로 활용된다.

In [ ]:
df_samsung[["Close","EWM_short","EWM_long"]].plot(figsize=(10,4), title="Exponential Moving Average")

In [ ]:
# 다양한 이동평균 비교하기
df_samsung[["Close", "CMA7", "EWM_short", "EWM_long"]].plot(figsize=(10,4))

### 4-3. resample

In [ ]:
df_samsung = stock.get_market_ohlcv_by_date("20220101", "20221021", "005930")
mon = df_samsung.resample('W').first()
print(mon)

In [ ]:
month_first = df_samsung.resample('MS').first()
month_first.head()

# 파라미터 참고: https://pandas.pydata.org/docs/reference/api/pandas.Series.resample.html
# https://summerorange.tistory.com/entry/Resample-%EC%8B%9C%EA%B0%84%EB%8C%80-%EC%9B%94-%EB%85%84-%EC%9D%BC%EB%B3%84-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EA%B0%92-%EA%B3%84%EC%82%B0